In [1]:
# load current YSA
from rdflib import Graph
ysa = Graph()
ysa.parse('http://finto.fi/rest/v1/ysa/data?format=text/turtle')
print(len(ysa))  # show number of triples

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7f46246e8a60>
Traceback (most recent call last):
  File "/home/oisuomin/git/Annif-corpora/tools/venv/lib/python3.10/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/home/oisuomin/git/Annif-corpora/tools/venv/lib/python3.10/site-packages/isodate/isodates.py", line 201, in parse_date
    return date(sign * int(groups['year']),
ValueError: month must be in 1..12


433048


In [2]:
# load current Allärs
allars = Graph()
allars.parse('http://finto.fi/rest/v1/allars/data?format=text/turtle')
print(len(allars))  # show number of triples

412097


In [3]:
# load current KAUNO (includes YSO)

kauno = Graph()
kauno.parse('../../Annif-corpora-restricted/kirjasampo/kauno-skos-reduced.ttl')
print(len(kauno))  # show number of triples

531574


In [4]:
import unicodedata

from rdflib import Literal, Namespace, URIRef
from rdflib.namespace import SKOS, OWL

YSO = Namespace('http://www.yso.fi/onto/yso/')
KAUNO = Namespace('http://www.yso.fi/onto/kauno/')
COMPLAIN = False  # whether to complain about unknown labels

def is_deprecated(uri):
    return (uri, OWL.deprecated, True) in kauno

def label_to_uris(label, source, voc, lang, complain=COMPLAIN):
    #print("looking up '{}' from {} in language {}".format(label, source, lang))
    # Remove trailing "." present in labels of some records
    value = Literal(unicodedata.normalize('NFC', label.rstrip('.')), lang)


    for prop in (SKOS.prefLabel, SKOS.altLabel):
        vocuri = voc.value(None, prop, value, any=True)
        if vocuri is not None:
            if vocuri.startswith(YSO) or vocuri.startswith(KAUNO):
                return [vocuri]
            for matchprop in (SKOS.exactMatch, SKOS.closeMatch):
                matches = [match for match in voc.objects(vocuri, matchprop)
                           if match.startswith(YSO)]
                if matches:
                    return matches

    # hackish fallbacks for cases like "kulttuuri", where YSO Cicero is out of date: look up via ysa/allars
    if source == 'yso/fin':
        matches = label_to_uris(label, "ysa", ysa, lang)
        if matches:
            #print("missing yso/fin label '{}' found via ysa".format(label))
            return matches

    if source == 'yso/swe':
        matches = label_to_uris(label, "allars", allars, lang)
        if matches:
            #print("missing yso/swe label '{}' found via allars".format(label))
            return matches
    
    if complain:
        print("Unknown label '{}' in source {}".format(label, source))
    return []


print(label_to_uris('kissa', 'yso/fin', kauno, 'fi'))  # YSO: kissa
print(label_to_uris('Ingmanin talo', 'yso/fin', kauno, 'fi')) # YSO: Casagranden talo
print(label_to_uris('siirtäminen', 'ysa', ysa, 'fi')) # YSO: siirto (liikuttaminen) + siirto (viestintä)
print(label_to_uris('lähioikeudet', 'yso/fin', kauno, 'fi')) # YSO: lähioikeudet
print(label_to_uris('kulttuuri', 'yso/fin', kauno, 'fi')) # YSO: kulttuuri
print(label_to_uris('Helsinki -- Kallio', 'ysa', ysa, 'fi'))  # YSO-paikat: Kallio (Helsinki)
print(label_to_uris('Zambia', 'kauno', kauno, 'fi'))  # YSO-paikat: Sambia
print(label_to_uris('not found', 'yso/fin', kauno, 'fi'))
print(label_to_uris('krimintataarit', 'kauno', kauno, 'fi'))  # KAUNO: Krimin tataarit
print(label_to_uris('sjöfästningar', 'bella', kauno, 'sv'))  # KAUNO: merilinnoitukset

[rdflib.term.URIRef('http://www.yso.fi/onto/kauno/p838')]
[rdflib.term.URIRef('http://www.yso.fi/onto/yso/p18095')]
[rdflib.term.URIRef('http://www.yso.fi/onto/yso/p5700')]
[rdflib.term.URIRef('http://www.yso.fi/onto/yso/p11910')]
[rdflib.term.URIRef('http://www.yso.fi/onto/kauno/p1794')]
[rdflib.term.URIRef('http://www.yso.fi/onto/yso/p105606')]
[]
[]
[rdflib.term.URIRef('http://www.yso.fi/onto/kauno/p6384')]
[rdflib.term.URIRef('http://www.yso.fi/onto/kauno/p6441')]


In [5]:
import functools

from rdflib.namespace import RDF, OWL, DCTERMS, SKOS

@functools.lru_cache(maxsize=30000)
def replace_concept(uri):
    replacement_candidates = list(kauno.objects(uri, DCTERMS.isReplacedBy)) + list(kauno.objects(uri, SKOS.narrowMatch))
    replacements = [rc for rc in replacement_candidates if rc.startswith(YSO)]
    if len(replacements) == 0:
        return [uri]
    else:
        print("replacing", uri, "with", ' '.join([str(r) for r in replacements]))
        if len(replacements) > 1:
            print("warning: multiple replacements for", uri)
        return replacements

@functools.lru_cache(maxsize=30000)
def check_concept(uri):
    if (uri, RDF.type, SKOS.Concept) not in kauno:
        print(str(uri), "not a skos:Concept")
        return False
    if kauno.value(uri, OWL.deprecated, None, any=True):
        print(str(uri), "is deprecated")
        return False
    return True

print(replace_concept(URIRef('http://www.yso.fi/onto/yso/p23766')))  # should be replaced by p1947 and p2203
print(replace_concept(URIRef('http://www.yso.fi/onto/kauno/p1779'))) # should be replaced by p1126

print(check_concept(URIRef('http://www.yso.fi/onto/yso/p23766')))    # is deprecated, should return False
print(check_concept(URIRef('http://www.yso.fi/onto/kauno/p1779')))   # is deprecated, should return False

[rdflib.term.URIRef('http://www.yso.fi/onto/yso/p23766')]
[rdflib.term.URIRef('http://www.yso.fi/onto/kauno/p1779')]
http://www.yso.fi/onto/yso/p23766 is deprecated
False
http://www.yso.fi/onto/kauno/p1779 is deprecated
False


In [6]:
import json

def get_subject_uris(subject_dicts_in):
    """Returns a list of subjects, i.e. strings extracted from the heading
    fields of the dictionaries in the input list."""

    subjects_out = []
    for subject_dict in subject_dicts_in:
        if ('source', 'kaunokki') in subject_dict.items():
            source = 'kaunokki'
            voc = kauno
            lang = 'fi'
        elif ('source', 'kaunokki/fin') in subject_dict.items():
            source = 'kaunokki/fin'
            voc = kauno
            lang = 'fi'
        elif ('source', 'kauno/fin') in subject_dict.items():
            source = 'kauno/fin'
            voc = kauno
            lang = 'fi'
        elif ('source', 'kauno/swe') in subject_dict.items():
            source = 'kauno/swe'
            voc = kauno
            lang = 'sv'
        elif ('source', 'bella') in subject_dict.items():
            source = 'bella'
            voc = kauno
            lang = 'sv'
        elif ('source', 'ysa') in subject_dict.items():
            source = 'ysa'
            voc = ysa
            lang = 'fi'
        elif ('source', 'allars') in subject_dict.items():
            source = 'allars'
            voc = allars
            lang = 'sv'
        elif ('source', 'yso/fin') in subject_dict.items():
            source = 'yso/fin'
            voc = kauno
            lang = 'fi'
        elif ('source', 'yso/swe') in subject_dict.items():
            source = 'yso/swe'
            voc = kauno
            lang = 'sv'
        else:
            continue
        
        uris = []
        
        if len(subject_dict['heading']) > 1:
            label = ' -- '.join(subject_dict['heading'])
            uris = label_to_uris(label, source, voc, lang, complain=False)

        if not uris:
            # not found as a precoordinated subject, try the individual parts instead
            uris = []
            for label in subject_dict['heading']:
                uris.extend(label_to_uris(label, source, voc, lang))
        
        for uri in uris:
            newuris = replace_concept(uri)
            for newuri in newuris:
                if check_concept(newuri):
                    subjects_out.append(newuri)

    return set(subjects_out)

def print_title_with_subject_uris(title, subjects):
    urilist = [label_to_uri(label, lang) for label, lang in subjects]
    urilist = [uri for uri in urilist if uri is not None]
    if not urilist:
        return  # no URIs found - omit the line
    print(unicodedata.normalize('NDC', title) + '\t' + '\t'.join(
         ['<'+item+'>' for sublist in urilist if sublist for item in sublist ]
    ))


def main(ndjson_in, output):
    """Prints the title (nimike) and subjects (aiheet) contained in the json
    objects of the input."""
    for ind, line in enumerate(ndjson_in):
        line_dict = json.loads(line)
        if 'title' not in line_dict:
            continue
        if 'subjectsExtended' not in line_dict:
            continue

        subjects = get_subject_uris(line_dict['subjectsExtended'])
        if subjects:
            print(line_dict['title'] + '\t' + '\t'.join((str(subj) for subj in subjects)), file=output)

In [7]:
import gzip

# the source files were created from finna-all-2023-05-01.ndjson.gz using the following method:
# 1. grep for '"source":"bella' or '"source":"kauno' (Note: no closing double quotes, to include e.g. kaunokki and kauno/fin) or 
# 2. split into language specific subsets using filter-by-language.sh
# This should be automated for the next round...

FINNA_BASE = '../../../proj/kauno-corpus/finna-source-kauno-or-bella'

In [8]:
%%time

with gzip.open(FINNA_BASE + '-swe.ndjson.gz', 'rt') as inputf:
    with gzip.open(FINNA_BASE + '-swe.tsv.gz', 'wt') as outputf:
        main(inputf, outputf)

http://www.yso.fi/onto/yso/p22100 is deprecated
http://www.yso.fi/onto/yso/p94277 not a skos:Concept
http://www.yso.fi/onto/yso/p106067 not a skos:Concept
http://www.yso.fi/onto/yso/p105111 not a skos:Concept
http://www.yso.fi/onto/yso/p105184 not a skos:Concept
http://www.yso.fi/onto/yso/p21256 is deprecated
http://www.yso.fi/onto/yso/p105663 not a skos:Concept
http://www.yso.fi/onto/yso/p105078 not a skos:Concept
http://www.yso.fi/onto/yso/p8641 is deprecated
http://www.yso.fi/onto/kauno/p3693 is deprecated
replacing http://www.yso.fi/onto/yso/p26380 with http://www.yso.fi/onto/yso/p370
http://www.yso.fi/onto/yso/p370 is deprecated
replacing http://www.yso.fi/onto/yso/p673 with http://www.yso.fi/onto/yso/p38073
http://www.yso.fi/onto/yso/p12262 is deprecated
http://www.yso.fi/onto/yso/p94383 not a skos:Concept
http://www.yso.fi/onto/yso/p4312 is deprecated
http://www.yso.fi/onto/yso/p4608 is deprecated
http://www.yso.fi/onto/yso/p18011 not a skos:Concept
http://www.yso.fi/onto/yso/p2

http://www.yso.fi/onto/yso/p116312 not a skos:Concept
http://www.yso.fi/onto/yso/p105280 not a skos:Concept
http://www.yso.fi/onto/yso/p106277 not a skos:Concept
http://www.yso.fi/onto/yso/p208084 not a skos:Concept
http://www.yso.fi/onto/yso/p20883 is deprecated
replacing http://www.yso.fi/onto/yso/p15459 with http://www.yso.fi/onto/yso/p2085
http://www.yso.fi/onto/yso/p94466 not a skos:Concept
http://www.yso.fi/onto/yso/p94341 not a skos:Concept
http://www.yso.fi/onto/yso/p2880 is deprecated
http://www.yso.fi/onto/yso/p26452 is deprecated
http://www.yso.fi/onto/kauno/p2805 is deprecated
http://www.yso.fi/onto/yso/p157464 not a skos:Concept
http://www.yso.fi/onto/yso/p107254 not a skos:Concept
http://www.yso.fi/onto/yso/p105354 not a skos:Concept
http://www.yso.fi/onto/yso/p105251 not a skos:Concept
http://www.yso.fi/onto/yso/p105612 not a skos:Concept
http://www.yso.fi/onto/yso/p2129 is deprecated
http://www.yso.fi/onto/yso/p9143 not a skos:Concept
http://www.yso.fi/onto/yso/p11535 n

http://www.yso.fi/onto/yso/p105850 not a skos:Concept
http://www.yso.fi/onto/yso/p107643 not a skos:Concept
http://www.yso.fi/onto/yso/p239385 not a skos:Concept
http://www.yso.fi/onto/yso/p226884 not a skos:Concept
http://www.yso.fi/onto/yso/p115950 not a skos:Concept
http://www.yso.fi/onto/yso/p16810 not a skos:Concept
http://www.yso.fi/onto/yso/p94126 not a skos:Concept
http://www.yso.fi/onto/yso/p105416 not a skos:Concept
http://www.yso.fi/onto/kauno/p1980 is deprecated
http://www.yso.fi/onto/yso/p115088 not a skos:Concept
http://www.yso.fi/onto/yso/p105659 not a skos:Concept
http://www.yso.fi/onto/yso/p111117 not a skos:Concept
http://www.yso.fi/onto/yso/p108437 not a skos:Concept
http://www.yso.fi/onto/yso/p106202 not a skos:Concept
http://www.yso.fi/onto/yso/p505667 not a skos:Concept
http://www.yso.fi/onto/yso/p234959 not a skos:Concept
http://www.yso.fi/onto/yso/p94284 not a skos:Concept
http://www.yso.fi/onto/kauno/p1698 is deprecated
http://www.yso.fi/onto/yso/p108007 not a 

http://www.yso.fi/onto/yso/p9880 is deprecated
http://www.yso.fi/onto/yso/p209905 not a skos:Concept
replacing http://www.yso.fi/onto/yso/p16559 with http://www.yso.fi/onto/yso/p1516
replacing http://www.yso.fi/onto/yso/p2322 with http://www.yso.fi/onto/yso/p2325
http://www.yso.fi/onto/yso/p8625 not a skos:Concept
http://www.yso.fi/onto/yso/p7259 not a skos:Concept
http://www.yso.fi/onto/yso/p14359 not a skos:Concept
http://www.yso.fi/onto/yso/p1044 not a skos:Concept
http://www.yso.fi/onto/yso/p3166 not a skos:Concept
http://www.yso.fi/onto/yso/p861 not a skos:Concept
http://www.yso.fi/onto/yso/p1148 not a skos:Concept
http://www.yso.fi/onto/yso/p107723 not a skos:Concept
http://www.yso.fi/onto/yso/p19531 is deprecated
http://www.yso.fi/onto/yso/p110694 not a skos:Concept
http://www.yso.fi/onto/yso/p105548 not a skos:Concept
http://www.yso.fi/onto/yso/p105029 not a skos:Concept
replacing http://www.yso.fi/onto/yso/p2448 with http://www.yso.fi/onto/yso/p8664
http://www.yso.fi/onto/yso/

http://www.yso.fi/onto/kauno/p5719 is deprecated
http://www.yso.fi/onto/yso/p106111 not a skos:Concept
http://www.yso.fi/onto/yso/p17984 not a skos:Concept
http://www.yso.fi/onto/yso/p106673 not a skos:Concept
http://www.yso.fi/onto/yso/p5729 not a skos:Concept
http://www.yso.fi/onto/yso/p798 not a skos:Concept
http://www.yso.fi/onto/yso/p533 not a skos:Concept
http://www.yso.fi/onto/yso/p796 not a skos:Concept
http://www.yso.fi/onto/yso/p6087 not a skos:Concept
http://www.yso.fi/onto/yso/p11615 not a skos:Concept
http://www.yso.fi/onto/yso/p108720 not a skos:Concept
http://www.yso.fi/onto/yso/p6787 not a skos:Concept
http://www.yso.fi/onto/yso/p8511 not a skos:Concept
http://www.yso.fi/onto/yso/p16056 not a skos:Concept
http://www.yso.fi/onto/yso/p67 not a skos:Concept
http://www.yso.fi/onto/yso/p19297 not a skos:Concept
http://www.yso.fi/onto/yso/p106044 not a skos:Concept
http://www.yso.fi/onto/yso/p106721 not a skos:Concept
http://www.yso.fi/onto/yso/p4477 not a skos:Concept
http:/

...

http://www.yso.fi/onto/yso/p20646 not a skos:Concept
http://www.yso.fi/onto/yso/p3835 not a skos:Concept
http://www.yso.fi/onto/yso/p2851 not a skos:Concept
http://www.yso.fi/onto/yso/p17556 not a skos:Concept
http://www.yso.fi/onto/yso/p105720 not a skos:Concept
http://www.yso.fi/onto/yso/p109331 not a skos:Concept
http://www.yso.fi/onto/yso/p1329 not a skos:Concept
http://www.yso.fi/onto/yso/p4403 not a skos:Concept
http://www.yso.fi/onto/yso/p1324 not a skos:Concept
http://www.yso.fi/onto/yso/p18049 not a skos:Concept
http://www.yso.fi/onto/yso/p1325 not a skos:Concept
http://www.yso.fi/onto/yso/p16071 not a skos:Concept
http://www.yso.fi/onto/yso/p17503 not a skos:Concept
replacing http://www.yso.fi/onto/yso/p1452 with http://www.yso.fi/onto/yso/p8090
http://www.yso.fi/onto/yso/p17754 not a skos:Concept
http://www.yso.fi/onto/yso/p5679 not a skos:Concept
http://www.yso.fi/onto/yso/p105096 not a skos:Concept
replacing http://www.yso.fi/onto/yso/p12813 with http://www.yso.fi/onto/yso

http://www.yso.fi/onto/yso/p10762 not a skos:Concept
http://www.yso.fi/onto/yso/p7902 not a skos:Concept
http://www.yso.fi/onto/yso/p7250 not a skos:Concept
http://www.yso.fi/onto/yso/p221 not a skos:Concept
http://www.yso.fi/onto/yso/p10759 not a skos:Concept
http://www.yso.fi/onto/yso/p1045 not a skos:Concept
http://www.yso.fi/onto/yso/p966 not a skos:Concept
http://www.yso.fi/onto/yso/p5629 not a skos:Concept
http://www.yso.fi/onto/yso/p5969 not a skos:Concept
http://www.yso.fi/onto/yso/p109167 not a skos:Concept
http://www.yso.fi/onto/yso/p120938 not a skos:Concept
http://www.yso.fi/onto/yso/p110900 not a skos:Concept
http://www.yso.fi/onto/yso/p15767 not a skos:Concept
http://www.yso.fi/onto/yso/p16156 not a skos:Concept
http://www.yso.fi/onto/yso/p5529 not a skos:Concept
http://www.yso.fi/onto/yso/p1017 not a skos:Concept
http://www.yso.fi/onto/yso/p4934 not a skos:Concept
http://www.yso.fi/onto/yso/p4999 not a skos:Concept
http://www.yso.fi/onto/yso/p163327 not a skos:Concept
ht

http://www.yso.fi/onto/yso/p12736 not a skos:Concept
http://www.yso.fi/onto/yso/p11413 not a skos:Concept
http://www.yso.fi/onto/yso/p9133 not a skos:Concept
http://www.yso.fi/onto/yso/p6639 not a skos:Concept
http://www.yso.fi/onto/yso/p129134 not a skos:Concept
http://www.yso.fi/onto/yso/p10963 not a skos:Concept
http://www.yso.fi/onto/yso/p632 not a skos:Concept
http://www.yso.fi/onto/yso/p2015 not a skos:Concept
http://www.yso.fi/onto/yso/p5004 not a skos:Concept
http://www.yso.fi/onto/yso/p6294 not a skos:Concept
http://www.yso.fi/onto/yso/p22537 not a skos:Concept
http://www.yso.fi/onto/yso/p108374 not a skos:Concept
http://www.yso.fi/onto/yso/p16342 is deprecated
http://www.yso.fi/onto/yso/p108403 not a skos:Concept
http://www.yso.fi/onto/yso/p11718 not a skos:Concept
http://www.yso.fi/onto/yso/p24752 not a skos:Concept
http://www.yso.fi/onto/yso/p18426 not a skos:Concept
http://www.yso.fi/onto/yso/p19498 not a skos:Concept
http://www.yso.fi/onto/yso/p8552 not a skos:Concept
htt

http://www.yso.fi/onto/yso/p1360 not a skos:Concept
http://www.yso.fi/onto/yso/p664 not a skos:Concept
http://www.yso.fi/onto/yso/p6662 not a skos:Concept
http://www.yso.fi/onto/yso/p5415 not a skos:Concept
http://www.yso.fi/onto/yso/p4490 not a skos:Concept
http://www.yso.fi/onto/yso/p4489 not a skos:Concept
http://www.yso.fi/onto/yso/p4979 not a skos:Concept
replacing http://www.yso.fi/onto/kauno/p6089 with http://www.yso.fi/onto/yso/p14
http://www.yso.fi/onto/yso/p1942 not a skos:Concept
http://www.yso.fi/onto/yso/p13352 not a skos:Concept
replacing http://www.yso.fi/onto/yso/p18234 with http://www.yso.fi/onto/yso/p4549
http://www.yso.fi/onto/yso/p2075 not a skos:Concept
http://www.yso.fi/onto/yso/p18409 not a skos:Concept
http://www.yso.fi/onto/yso/p4902 not a skos:Concept
replacing http://www.yso.fi/onto/yso/p24131 with http://www.yso.fi/onto/yso/p15211
http://www.yso.fi/onto/yso/p107669 not a skos:Concept
http://www.yso.fi/onto/yso/p8289 not a skos:Concept
http://www.yso.fi/onto/

http://www.yso.fi/onto/yso/p387 not a skos:Concept
http://www.yso.fi/onto/yso/p107683 not a skos:Concept
http://www.yso.fi/onto/yso/p6793 not a skos:Concept
http://www.yso.fi/onto/yso/p15487 not a skos:Concept
replacing http://www.yso.fi/onto/kauno/p6476 with http://www.yso.fi/onto/yso/p605
http://www.yso.fi/onto/yso/p131239 not a skos:Concept
http://www.yso.fi/onto/yso/p3536 not a skos:Concept
http://www.yso.fi/onto/yso/p3528 not a skos:Concept
http://www.yso.fi/onto/yso/p5504 not a skos:Concept
http://www.yso.fi/onto/yso/p105717 not a skos:Concept
replacing http://www.yso.fi/onto/yso/p14631 with http://www.yso.fi/onto/yso/p14630
http://www.yso.fi/onto/yso/p107738 not a skos:Concept
http://www.yso.fi/onto/yso/p108565 not a skos:Concept
http://www.yso.fi/onto/yso/p2138 not a skos:Concept
http://www.yso.fi/onto/yso/p5619 not a skos:Concept
http://www.yso.fi/onto/yso/p19775 is deprecated
http://www.yso.fi/onto/yso/p929 not a skos:Concept
http://www.yso.fi/onto/yso/p11062 not a skos:Conce

In [9]:
%%time

with gzip.open(FINNA_BASE + '-eng.ndjson.gz', 'rt') as inputf:
    with gzip.open(FINNA_BASE + '-eng.tsv.gz', 'wt') as outputf:
        main(inputf, outputf)

http://www.yso.fi/onto/yso/p505266 not a skos:Concept
http://www.yso.fi/onto/yso/p5161 not a skos:Concept
http://www.yso.fi/onto/yso/p107650 not a skos:Concept
http://www.yso.fi/onto/yso/p7386 is deprecated
http://www.yso.fi/onto/yso/p203172 not a skos:Concept
http://www.yso.fi/onto/yso/p19707 not a skos:Concept
http://www.yso.fi/onto/yso/p21825 not a skos:Concept
http://www.yso.fi/onto/kauno/p6579 is deprecated
http://www.yso.fi/onto/yso/p14718 not a skos:Concept
http://www.yso.fi/onto/yso/p17281 not a skos:Concept
http://www.yso.fi/onto/yso/p25686 not a skos:Concept
http://www.yso.fi/onto/yso/p21628 not a skos:Concept
http://www.yso.fi/onto/yso/p23732 not a skos:Concept
http://www.yso.fi/onto/yso/p166195 not a skos:Concept
http://www.yso.fi/onto/yso/p6343 not a skos:Concept
http://www.yso.fi/onto/yso/p234580 not a skos:Concept
http://www.yso.fi/onto/yso/p149166 not a skos:Concept
http://www.yso.fi/onto/yso/p22705 not a skos:Concept
http://www.yso.fi/onto/yso/p26239 not a skos:Concept

http://www.yso.fi/onto/yso/p506096 not a skos:Concept
http://www.yso.fi/onto/yso/p25685 not a skos:Concept
http://www.yso.fi/onto/yso/p105976 not a skos:Concept
http://www.yso.fi/onto/yso/p22706 not a skos:Concept
http://www.yso.fi/onto/yso/p2919 not a skos:Concept
http://www.yso.fi/onto/yso/p18327 not a skos:Concept
http://www.yso.fi/onto/yso/p6727 not a skos:Concept
http://www.yso.fi/onto/yso/p17168 not a skos:Concept
http://www.yso.fi/onto/yso/p22360 not a skos:Concept
http://www.yso.fi/onto/yso/p9002 not a skos:Concept
http://www.yso.fi/onto/yso/p36 not a skos:Concept
http://www.yso.fi/onto/yso/p4551 not a skos:Concept
http://www.yso.fi/onto/yso/p20347 not a skos:Concept
http://www.yso.fi/onto/yso/p2221 not a skos:Concept
http://www.yso.fi/onto/yso/p19045 not a skos:Concept
http://www.yso.fi/onto/kauno/p1453 is deprecated
http://www.yso.fi/onto/yso/p108077 not a skos:Concept
http://www.yso.fi/onto/yso/p118609 not a skos:Concept
http://www.yso.fi/onto/yso/p239422 not a skos:Concept


http://www.yso.fi/onto/yso/p5668 not a skos:Concept
http://www.yso.fi/onto/yso/p145 not a skos:Concept
http://www.yso.fi/onto/yso/p107651 not a skos:Concept
http://www.yso.fi/onto/yso/p2419 not a skos:Concept
http://www.yso.fi/onto/yso/p2416 not a skos:Concept
http://www.yso.fi/onto/yso/p1090 not a skos:Concept
http://www.yso.fi/onto/yso/p109299 not a skos:Concept
http://www.yso.fi/onto/yso/p166490 not a skos:Concept
http://www.yso.fi/onto/kauno/p3321 is deprecated
http://www.yso.fi/onto/yso/p15377 not a skos:Concept
http://www.yso.fi/onto/yso/p29160 not a skos:Concept
http://www.yso.fi/onto/yso/p27833 not a skos:Concept
http://www.yso.fi/onto/yso/p22724 not a skos:Concept
http://www.yso.fi/onto/yso/p3482 not a skos:Concept
http://www.yso.fi/onto/yso/p8629 not a skos:Concept
http://www.yso.fi/onto/yso/p25413 not a skos:Concept
http://www.yso.fi/onto/kauno/p401 is deprecated
http://www.yso.fi/onto/yso/p94418 not a skos:Concept
replacing http://www.yso.fi/onto/yso/p2375 with http://www.y

http://www.yso.fi/onto/yso/p20257 not a skos:Concept
http://www.yso.fi/onto/kauno/p2646 is deprecated
http://www.yso.fi/onto/yso/p108681 not a skos:Concept
http://www.yso.fi/onto/yso/p11455 not a skos:Concept
http://www.yso.fi/onto/yso/p1439 not a skos:Concept
http://www.yso.fi/onto/yso/p16512 not a skos:Concept
http://www.yso.fi/onto/yso/p19 not a skos:Concept
replacing http://www.yso.fi/onto/yso/p614 with http://www.yso.fi/onto/yso/p26592
http://www.yso.fi/onto/yso/p155167 not a skos:Concept
http://www.yso.fi/onto/yso/p8875 not a skos:Concept
http://www.yso.fi/onto/yso/p2609 not a skos:Concept
http://www.yso.fi/onto/yso/p2870 not a skos:Concept
http://www.yso.fi/onto/yso/p24028 not a skos:Concept
http://www.yso.fi/onto/yso/p209155 not a skos:Concept
http://www.yso.fi/onto/yso/p17596 not a skos:Concept
http://www.yso.fi/onto/yso/p2616 not a skos:Concept
http://www.yso.fi/onto/yso/p25473 not a skos:Concept
http://www.yso.fi/onto/yso/p7005 not a skos:Concept
http://www.yso.fi/onto/yso/p

http://www.yso.fi/onto/yso/p25933 not a skos:Concept
http://www.yso.fi/onto/yso/p3589 not a skos:Concept
http://www.yso.fi/onto/yso/p108180 not a skos:Concept
http://www.yso.fi/onto/yso/p20067 not a skos:Concept
http://www.yso.fi/onto/yso/p6828 not a skos:Concept
http://www.yso.fi/onto/yso/p105105 not a skos:Concept
http://www.yso.fi/onto/yso/p25160 not a skos:Concept
http://www.yso.fi/onto/yso/p109329 not a skos:Concept
http://www.yso.fi/onto/yso/p3507 not a skos:Concept
http://www.yso.fi/onto/yso/p29699 not a skos:Concept
http://www.yso.fi/onto/yso/p25577 not a skos:Concept
http://www.yso.fi/onto/yso/p7174 not a skos:Concept
http://www.yso.fi/onto/yso/p19863 not a skos:Concept
http://www.yso.fi/onto/yso/p94382 not a skos:Concept
http://www.yso.fi/onto/yso/p20925 not a skos:Concept
http://www.yso.fi/onto/yso/p113324 not a skos:Concept
http://www.yso.fi/onto/yso/p2291 not a skos:Concept
http://www.yso.fi/onto/yso/p7750 not a skos:Concept
http://www.yso.fi/onto/yso/p212484 not a skos:Co

In [10]:
%%time

with gzip.open(FINNA_BASE + '-fin.ndjson.gz', 'rt') as inputf:
    with gzip.open(FINNA_BASE + '-fin.tsv.gz', 'wt') as outputf:
        main(inputf, outputf)

http://www.yso.fi/onto/yso/p7461 not a skos:Concept
http://www.yso.fi/onto/yso/p13915 not a skos:Concept
http://www.yso.fi/onto/yso/p105072 not a skos:Concept
http://www.yso.fi/onto/yso/p108002 not a skos:Concept
http://www.yso.fi/onto/yso/p9048 not a skos:Concept
http://www.yso.fi/onto/yso/p94443 not a skos:Concept
http://www.yso.fi/onto/yso/p19920 not a skos:Concept
http://www.yso.fi/onto/yso/p94496 not a skos:Concept
http://www.yso.fi/onto/yso/p3027 is deprecated
http://www.yso.fi/onto/yso/p94348 not a skos:Concept
http://www.yso.fi/onto/yso/p94401 not a skos:Concept
http://www.yso.fi/onto/yso/p14979 not a skos:Concept
http://www.yso.fi/onto/yso/p8999 not a skos:Concept
http://www.yso.fi/onto/yso/p132522 not a skos:Concept
http://www.yso.fi/onto/yso/p106323 not a skos:Concept
http://www.yso.fi/onto/yso/p10969 not a skos:Concept
http://www.yso.fi/onto/yso/p25898 not a skos:Concept
http://www.yso.fi/onto/yso/p105485 not a skos:Concept
http://www.yso.fi/onto/yso/p164913 not a skos:Conc

http://www.yso.fi/onto/yso/p94273 not a skos:Concept
http://www.yso.fi/onto/yso/p94143 not a skos:Concept
http://www.yso.fi/onto/yso/p1132 not a skos:Concept
http://www.yso.fi/onto/yso/p8575 not a skos:Concept
http://www.yso.fi/onto/yso/p20121 not a skos:Concept
http://www.yso.fi/onto/yso/p5617 not a skos:Concept
http://www.yso.fi/onto/yso/p10919 not a skos:Concept
http://www.yso.fi/onto/yso/p21183 not a skos:Concept
http://www.yso.fi/onto/yso/p8378 not a skos:Concept
http://www.yso.fi/onto/yso/p2323 not a skos:Concept
http://www.yso.fi/onto/yso/p3394 not a skos:Concept
http://www.yso.fi/onto/yso/p94414 not a skos:Concept
http://www.yso.fi/onto/yso/p107802 not a skos:Concept
http://www.yso.fi/onto/yso/p94182 not a skos:Concept
http://www.yso.fi/onto/yso/p11703 not a skos:Concept
http://www.yso.fi/onto/yso/p94404 not a skos:Concept
http://www.yso.fi/onto/yso/p106561 not a skos:Concept
http://www.yso.fi/onto/yso/p10587 not a skos:Concept
http://www.yso.fi/onto/yso/p14031 not a skos:Conce

http://www.yso.fi/onto/yso/p14074 not a skos:Concept
http://www.yso.fi/onto/yso/p148035 not a skos:Concept
http://www.yso.fi/onto/yso/p123967 not a skos:Concept
http://www.yso.fi/onto/yso/p107725 not a skos:Concept
http://www.yso.fi/onto/yso/p217568 not a skos:Concept
http://www.yso.fi/onto/yso/p8122 not a skos:Concept
http://www.yso.fi/onto/yso/p8818 not a skos:Concept
http://www.yso.fi/onto/yso/p20630 not a skos:Concept
http://www.yso.fi/onto/yso/p8175 not a skos:Concept
http://www.yso.fi/onto/yso/p109095 not a skos:Concept
http://www.yso.fi/onto/yso/p94178 not a skos:Concept
http://www.yso.fi/onto/yso/p3301 not a skos:Concept
http://www.yso.fi/onto/yso/p105874 not a skos:Concept
http://www.yso.fi/onto/yso/p13354 not a skos:Concept
http://www.yso.fi/onto/yso/p80 not a skos:Concept
http://www.yso.fi/onto/yso/p5897 not a skos:Concept
http://www.yso.fi/onto/yso/p19930 not a skos:Concept
http://www.yso.fi/onto/yso/p22541 not a skos:Concept
http://www.yso.fi/onto/yso/p9833 not a skos:Conc

http://www.yso.fi/onto/yso/p2289 not a skos:Concept
http://www.yso.fi/onto/kauno/p667 is deprecated
http://www.yso.fi/onto/yso/p24825 not a skos:Concept
http://www.yso.fi/onto/yso/p13026 not a skos:Concept
http://www.yso.fi/onto/yso/p14638 not a skos:Concept
http://www.yso.fi/onto/yso/p9380 not a skos:Concept
http://www.yso.fi/onto/yso/p2856 not a skos:Concept
http://www.yso.fi/onto/yso/p6118 not a skos:Concept
http://www.yso.fi/onto/yso/p9405 not a skos:Concept
http://www.yso.fi/onto/yso/p7176 is deprecated
http://www.yso.fi/onto/yso/p94499 not a skos:Concept
http://www.yso.fi/onto/yso/p7662 not a skos:Concept
http://www.yso.fi/onto/yso/p2658 not a skos:Concept
http://www.yso.fi/onto/yso/p16432 not a skos:Concept
http://www.yso.fi/onto/yso/p18782 not a skos:Concept
http://www.yso.fi/onto/yso/p13191 not a skos:Concept
replacing http://www.yso.fi/onto/yso/p6012 with http://www.yso.fi/onto/yso/p123996
http://www.yso.fi/onto/yso/p123996 not a skos:Concept
http://www.yso.fi/onto/yso/p94129

http://www.yso.fi/onto/yso/p19001 not a skos:Concept
http://www.yso.fi/onto/yso/p6673 not a skos:Concept
http://www.yso.fi/onto/yso/p1725 not a skos:Concept
http://www.yso.fi/onto/yso/p205603 not a skos:Concept
http://www.yso.fi/onto/yso/p107671 not a skos:Concept
http://www.yso.fi/onto/yso/p1795 not a skos:Concept
http://www.yso.fi/onto/yso/p10997 not a skos:Concept
http://www.yso.fi/onto/yso/p117190 not a skos:Concept
http://www.yso.fi/onto/yso/p3951 not a skos:Concept
http://www.yso.fi/onto/yso/p177 not a skos:Concept
http://www.yso.fi/onto/yso/p4692 not a skos:Concept
http://www.yso.fi/onto/yso/p5516 not a skos:Concept
http://www.yso.fi/onto/yso/p16229 not a skos:Concept
http://www.yso.fi/onto/yso/p7717 not a skos:Concept
http://www.yso.fi/onto/yso/p9602 not a skos:Concept
replacing http://www.yso.fi/onto/yso/p2450 with http://www.yso.fi/onto/yso/p7721
http://www.yso.fi/onto/yso/p94218 not a skos:Concept
http://www.yso.fi/onto/yso/p14572 not a skos:Concept
http://www.yso.fi/onto/ys

...

http://www.yso.fi/onto/yso/p22575 not a skos:Concept
http://www.yso.fi/onto/yso/p24504 not a skos:Concept
http://www.yso.fi/onto/yso/p1028 not a skos:Concept
http://www.yso.fi/onto/yso/p1545 not a skos:Concept
http://www.yso.fi/onto/yso/p234566 not a skos:Concept
http://www.yso.fi/onto/yso/p105603 not a skos:Concept
http://www.yso.fi/onto/yso/p10402 not a skos:Concept
http://www.yso.fi/onto/yso/p109637 not a skos:Concept
http://www.yso.fi/onto/yso/p16840 not a skos:Concept
http://www.yso.fi/onto/yso/p2987 not a skos:Concept
http://www.yso.fi/onto/yso/p16304 not a skos:Concept
http://www.yso.fi/onto/yso/p112007 not a skos:Concept
http://www.yso.fi/onto/yso/p105389 not a skos:Concept
replacing http://www.yso.fi/onto/yso/p10623 with http://www.yso.fi/onto/yso/p4268
http://www.yso.fi/onto/yso/p94429 not a skos:Concept
http://www.yso.fi/onto/yso/p108084 not a skos:Concept
http://www.yso.fi/onto/yso/p12783 not a skos:Concept
http://www.yso.fi/onto/yso/p22769 not a skos:Concept
http://www.yso

http://www.yso.fi/onto/yso/p23199 not a skos:Concept
http://www.yso.fi/onto/yso/p109455 not a skos:Concept
http://www.yso.fi/onto/yso/p3816 not a skos:Concept
http://www.yso.fi/onto/yso/p109270 not a skos:Concept
http://www.yso.fi/onto/yso/p19688 not a skos:Concept
http://www.yso.fi/onto/yso/p27756 not a skos:Concept
http://www.yso.fi/onto/yso/p26040 not a skos:Concept
http://www.yso.fi/onto/yso/p10902 not a skos:Concept
http://www.yso.fi/onto/yso/p5571 not a skos:Concept
http://www.yso.fi/onto/yso/p23403 not a skos:Concept
http://www.yso.fi/onto/yso/p7050 not a skos:Concept
http://www.yso.fi/onto/yso/p12639 not a skos:Concept
http://www.yso.fi/onto/yso/p4840 not a skos:Concept
replacing http://www.yso.fi/onto/yso/p4566 with http://www.yso.fi/onto/yso/p180
http://www.yso.fi/onto/yso/p20772 not a skos:Concept
http://www.yso.fi/onto/yso/p20884 not a skos:Concept
http://www.yso.fi/onto/yso/p20671 not a skos:Concept
http://www.yso.fi/onto/yso/p6293 not a skos:Concept
http://www.yso.fi/onto

http://www.yso.fi/onto/yso/p179675 not a skos:Concept
http://www.yso.fi/onto/yso/p17508 not a skos:Concept
http://www.yso.fi/onto/yso/p7664 not a skos:Concept
http://www.yso.fi/onto/yso/p110099 not a skos:Concept
http://www.yso.fi/onto/yso/p28783 not a skos:Concept
http://www.yso.fi/onto/yso/p94208 not a skos:Concept
http://www.yso.fi/onto/yso/p219296 not a skos:Concept
http://www.yso.fi/onto/yso/p2346 not a skos:Concept
http://www.yso.fi/onto/yso/p3391 not a skos:Concept
http://www.yso.fi/onto/yso/p1730 not a skos:Concept
replacing http://www.yso.fi/onto/yso/p5330 with http://www.yso.fi/onto/yso/p26448
http://www.yso.fi/onto/yso/p128092 not a skos:Concept
http://www.yso.fi/onto/yso/p21800 not a skos:Concept
http://www.yso.fi/onto/yso/p23952 not a skos:Concept
http://www.yso.fi/onto/yso/p1957 not a skos:Concept
http://www.yso.fi/onto/yso/p194896 not a skos:Concept
http://www.yso.fi/onto/yso/p24107 not a skos:Concept
http://www.yso.fi/onto/yso/p1723 not a skos:Concept
replacing http://w

http://www.yso.fi/onto/yso/p2621 not a skos:Concept
http://www.yso.fi/onto/yso/p2370 not a skos:Concept
http://www.yso.fi/onto/yso/p160717 not a skos:Concept
http://www.yso.fi/onto/yso/p22256 not a skos:Concept
http://www.yso.fi/onto/yso/p207558 not a skos:Concept
http://www.yso.fi/onto/yso/p15299 is deprecated
http://www.yso.fi/onto/yso/p19082 is deprecated
http://www.yso.fi/onto/yso/p9665 not a skos:Concept
http://www.yso.fi/onto/yso/p105826 not a skos:Concept
http://www.yso.fi/onto/yso/p26599 not a skos:Concept
http://www.yso.fi/onto/yso/p8292 not a skos:Concept
http://www.yso.fi/onto/yso/p22221 not a skos:Concept
http://www.yso.fi/onto/yso/p106142 not a skos:Concept
http://www.yso.fi/onto/yso/p108896 not a skos:Concept
http://www.yso.fi/onto/yso/p205678 not a skos:Concept
http://www.yso.fi/onto/yso/p106166 not a skos:Concept
http://www.yso.fi/onto/yso/p105862 not a skos:Concept
http://www.yso.fi/onto/yso/p94189 not a skos:Concept
http://www.yso.fi/onto/yso/p94261 not a skos:Concept

http://www.yso.fi/onto/yso/p1693 not a skos:Concept
http://www.yso.fi/onto/yso/p110630 not a skos:Concept
http://www.yso.fi/onto/yso/p26389 not a skos:Concept
http://www.yso.fi/onto/yso/p5438 not a skos:Concept
http://www.yso.fi/onto/yso/p5862 not a skos:Concept
http://www.yso.fi/onto/yso/p22834 not a skos:Concept
http://www.yso.fi/onto/yso/p26902 not a skos:Concept
http://www.yso.fi/onto/yso/p7772 not a skos:Concept
http://www.yso.fi/onto/yso/p129133 not a skos:Concept
http://www.yso.fi/onto/yso/p16608 not a skos:Concept
http://www.yso.fi/onto/yso/p18341 not a skos:Concept
http://www.yso.fi/onto/yso/p94397 not a skos:Concept
http://www.yso.fi/onto/yso/p740 not a skos:Concept
http://www.yso.fi/onto/yso/p24426 not a skos:Concept
http://www.yso.fi/onto/yso/p11336 not a skos:Concept
http://www.yso.fi/onto/yso/p26685 not a skos:Concept
http://www.yso.fi/onto/yso/p162942 not a skos:Concept
http://www.yso.fi/onto/yso/p18082 not a skos:Concept
http://www.yso.fi/onto/yso/p107955 not a skos:Con